In [12]:
import json
import requests
from bs4 import BeautifulSoup
import pandas as pd
import time

In [13]:
links = None

with open('links.json', 'r') as f:
    links = json.loads(f.read())

links

{'Hindi-Persian': 'https://en.m.wiktionary.org/wiki/Category:Hindi_terms_borrowed_from_Persian'}

In [14]:
base_link = "https://en.m.wiktionary.org"


In [17]:
def get_words_ipas(pair, link, page_num):
    page = requests.get(link)
    soup = BeautifulSoup(page.content, 'html.parser')
    borrowed_words = []

    res = soup.find_all("div", class_="mw-category-group")
    try:

        for i, r in enumerate(res):
            if i > 0:
                list_words = r.find_all('a')
                for word in list_words:
                    # print(word.text, base_link + word['href'])
                    ipas = []
                    ipa_page = requests.get(base_link + word['href'])
                    ipa_soup = BeautifulSoup(ipa_page.content, 'html.parser')

                    for ipa in ipa_soup.find_all("span", {'class': 'IPA'}):
                        txt = ipa.text
                        if (txt.startswith('/') or txt.startswith('[')) and (txt.endswith('/') or txt.endswith(']')):
                            ipas.append(ipa.text)
                            # print(word.text, ipa.text)  
                    borrowed_words.append([word.text, ipas])

        print(pair, page_num, "done")
        next_link = soup.find(
            lambda tag: tag.name == 'a' and 'next page' in tag.text.lower()
        )
        if next_link != None:
            more_words = get_words_ipas(pair, base_link + next_link['href'], page_num+1)
            borrowed_words = borrowed_words + more_words
    except:
        return borrowed_words
        
    return borrowed_words

In [18]:
for pair in links:
    words = get_words_ipas(pair, links[pair], 1)
    df = pd.DataFrame(words, columns=['word', 'ipas'])
    df.to_csv("results/{}.csv".format(pair), index=False)
    print(pair, "done")

Hindi-Persian 1 done
Hindi-Persian 2 done
Hindi-Persian 3 done
Hindi-Persian 4 done
Hindi-Persian 5 done
Hindi-Persian 6 done
Hindi-Persian 7 done
Hindi-Persian done
